In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt

import autograd.numpy as np
import autograd.numpy.random as npr

from autograd import grad
from autograd.misc.optimizers import adam

import pandas as pd
from pandas import DataFrame

import os

In [2]:
with open('../data/state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print (len(state_features))

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission', 'bloc']
48


In [3]:
df_train = pd.read_csv('../data/rl_train_set_unscaled.csv')
df_val = pd.read_csv('../data/rl_val_set_unscaled.csv')
df_test = pd.read_csv('../data/rl_test_set_unscaled.csv')

In [4]:
# let the mortality labels  be -1 and 1: -1 for survival
df_train['died_in_hosp'][df_train['died_in_hosp'] == 0] = -1
df_val['died_in_hosp'][df_val['died_in_hosp'] == 0] = -1
df_test['died_in_hosp'][df_test['died_in_hosp'] == 0] = -1

/afs/csail.mit.edu/u/a/araghu/.conda/envs/my_root/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/afs/csail.mit.edu/u/a/araghu/.conda/envs/my_root/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/afs/csail.mit.edu/u/a/araghu/.conda/envs/my_root/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [5]:
target_feat = list(np.loadtxt('../data/state_features_pred.txt', dtype=str))
target_feat.append('died_in_hosp')

cur_feat = list(np.loadtxt('../data/state_features.txt', dtype=str))
cur_feat.append('iv_input_norm')
cur_feat.append('vaso_input_norm')

In [6]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [7]:
inv_action_map = {}
for iv in range(5):
    for vaso in range(5):
        inv_action_map[5*iv+vaso] = [iv,vaso]    

In [8]:
iv_mean = df_train['iv_input'].mean()
iv_std = df_train['iv_input'].std()

vaso_mean = df_train['vaso_input'].mean()
vaso_std = df_train['vaso_input'].std()

In [9]:
df_train['iv_input_norm'] = (df_train['iv_input']- iv_mean)/iv_std
df_train['vaso_input_norm'] = (df_train['vaso_input'] - vaso_mean)/vaso_std

df_val['iv_input_norm'] = (df_val['iv_input']- iv_mean)/iv_std
df_val['vaso_input_norm'] = (df_val['vaso_input'] - vaso_mean)/vaso_std

df_test['iv_input_norm'] = (df_test['iv_input']- iv_mean)/iv_std
df_test['vaso_input_norm'] = (df_test['vaso_input'] - vaso_mean)/vaso_std

In [10]:
#  X: (states, actions)
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
def make_data(df_in):
    X = []
    Y = []
    for count,i in enumerate(df_in.index):
        cur_state = df_in.loc[i,cur_feat]
        if i != df_in.index[-1]:
            # if not terminal step in trajectory             
            if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
                target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
                target[-1] = df_in.loc[i, 'died_in_hosp']
                Y.append(target)
                X.append(cur_state)

        if count % 10000 == 0 and count > 0:
            print(count)

    return np.array(X),np.array(Y)

In [11]:
#  X: (s_t-3, a_t-3, s_t-2, a_t-2,s_t-1, a_t-1, s_t, a_t )
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
hist = 3
def make_data_history(df_in):
    df_in = df_in.reset_index()
    X = []
    Y = []
    count_in_traj = 0
    for count,i in enumerate(df_in.index):
        if count % 10000 == 0 and count > 0:
            print (count)
        
        # skip the last one; no next state
        if i == df_in.index[-1]:
            break
       
        # if not terminal step in trajectory    
        if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
            count_in_traj += 1
            if count_in_traj >=(hist+1):

                target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
                target[-1] = df_in.loc[i, 'died_in_hosp']
                Y.append(target)
                state = df_in.loc[i-hist, cur_feat]
                for index in range(hist-1,-1,-1):
                    state = np.hstack([state,df_in.loc[i-index, cur_feat]])
                #state = np.hstack([df_in.loc[i-2, cur_feat], df_in.loc[i-1, cur_feat], df_in.loc[i, cur_feat]])
                X.append(state)
            else:
                continue
        else:
            count_in_traj = 0

    return np.array(X),np.array(Y)

In [12]:
#  X: (s_t-3, a_t-3, s_t-2, a_t-2,s_t-1, a_t-1, s_t, a_t )
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
hist = 3
def make_data_history_zeros(df_in):
    df_in = df_in.reset_index()
    X = []
    Y = []
    count_in_traj = 0
    for count,i in enumerate(df_in.index):
        if count % 10000 == 0 and count > 0:
            print (count)
        
        # skip the last one; no next state
        if i == df_in.index[-1]:
            break
       
        # if not terminal step in trajectory    
        if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
            count_in_traj += 1
            target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
            target[-1] = df_in.loc[i, 'died_in_hosp']
            Y.append(target)
            if count_in_traj >=(hist+1):
                state = df_in.loc[i-hist, cur_feat]
                for index in range(hist-1,-1,-1):
                    state = np.hstack([state,df_in.loc[i-index, cur_feat]])
                X.append(state)
            else:
                num_zeros = (hist+1) - count_in_traj
                num_actual = count_in_traj - 1
                state = np.hstack([np.zeros(len(cur_feat)) for _ in range(num_zeros)])
                for index in range(num_actual, 0, -1):
                    state = np.hstack([state,df_in.loc[i-index, cur_feat]])
                state = np.hstack([state, df_in.loc[i, cur_feat]])
                X.append(state) 
        else:
            count_in_traj = 0

    return np.array(X),np.array(Y)

In [13]:
# import os
# dire = 'converted_data/'
# if not os.path.exists(dire):
#     os.makedirs(dire)
    
# if not os.path.exists(dire + 'X_train.txt'):
#     x_train_nohist, y_train_nohist = make_data(df_train)
#     np.savetxt(dire + 'X_train.txt',x_train_nohist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_train.txt',y_train_nohist,fmt='%5.4f')
#     print("Saved train")
# else:
#     x_train_nohist = np.loadtxt(dire + 'X_train.txt')
#     y_train_nohist = np.loadtxt(dire + 'Y_train.txt')
#     print ("Loaded train")

# if not os.path.exists(dire + 'X_val.txt'):
#     x_val_nohist,y_val_nohist = make_data(df_val)
#     np.savetxt(dire + 'X_val.txt',x_val_nohist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_val.txt',y_val_nohist,fmt='%5.4f')
#     print ("Saved val")
# else:
#     x_val_nohist = np.loadtxt(dire + 'X_val.txt')
#     y_val_nohist = np.loadtxt(dire + 'Y_val.txt')
#     print ("Loaded val")
    
# if not os.path.exists(dire + 'X_test.txt'):
#     x_test_nohist, y_test_nohist = make_data(df_test)
#     np.savetxt(dire + 'X_test.txt',x_test_nohist,fmt='%5.4f')
#     np.savetxt(dire +    'Y_test.txt',y_test_nohist,fmt='%5.4f')
#     print ("Saved test")
# else:
#     x_test_nohist = np.loadtxt(dire + 'X_test.txt')
#     y_test_nohist = np.loadtxt(dire + 'Y_test.txt')
#     print ("Loaded test")

In [14]:
# dire = 'converted_data/'
# if not os.path.exists(dire):
#     os.makedirs(dire)
    
# if not os.path.exists(dire + 'X_train_hist.txt'):
#     x_train_hist, y_train_hist = make_data_history(df_train)
#     np.savetxt(dire + 'X_train_hist.txt',x_train_hist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_train_hist.txt',y_train_hist,fmt='%5.4f')
#     print ("Saved train")
# else:
#     x_train_hist = np.loadtxt(dire + 'X_train_hist.txt')
#     y_train_hist = np.loadtxt(dire + 'Y_train_hist.txt')
#     print ("Loaded train")

# if not os.path.exists(dire + 'X_val_hist.txt'):
#     x_val_hist,y_val_hist = make_data_history(df_val)
#     np.savetxt(dire + 'X_val_hist.txt',x_val_hist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_val_hist.txt',y_val_hist,fmt='%5.4f')
#     print ("Saved val")
# else:
#     x_val_hist = np.loadtxt(dire + 'X_val_hist.txt')
#     y_val_hist = np.loadtxt(dire + 'Y_val_hist.txt')
#     print ("Loaded val")
    
# if not os.path.exists(dire + 'X_test_hist.txt'):
#     x_test_hist, y_test_hist = make_data_history(df_test)
#     np.savetxt(dire + 'X_test_hist.txt',x_test_hist,fmt='%5.4f')
#     np.savetxt(dire + 'Y_test_hist.txt',y_test_hist,fmt='%5.4f')
#     print ("Saved test")
# else:
#     x_test_hist = np.loadtxt(dire + 'X_test_hist.txt')
#     y_test_hist = np.loadtxt(dire + 'Y_test_hist.txt')
#     print ("Loaded test")

In [15]:
dire = 'converted_data/'
if not os.path.exists(dire):
    os.makedirs(dire)

if not os.path.exists(dire + 'X_train_hist_zeros.txt'):
    train_feat_zeros, train_labels_zeros = make_data_history_zeros(df_train)
    np.savetxt(dire + 'X_train_hist_zeros.txt',train_feat_zeros,fmt='%5.4f')
    np.savetxt(dire + 'Y_train_hist_zeros.txt',train_labels_zeros,fmt='%5.4f')
    print ("Saved train_zeros")
else:
    train_feat_zeros = np.loadtxt(dire + 'X_train_hist_zeros.txt')
    train_labels_zeros = np.loadtxt(dire + 'Y_train_hist_zeros.txt')
    print ("Loaded train_zeros")

if not os.path.exists(dire + 'X_val_hist_zeros.txt'):
    val_feat_zeros, val_labels_zeros = make_data_history_zeros(df_val)
    np.savetxt(dire + 'X_val_hist_zeros.txt',val_feat_zeros,fmt='%5.4f')
    np.savetxt(dire + 'Y_val_hist_zeros.txt',val_labels_zeros,fmt='%5.4f')
    print ("Saved val_zeros")
else:
    val_feat_zeros = np.loadtxt(dire + 'X_val_hist_zeros.txt')
    val_labels_zeros = np.loadtxt(dire + 'Y_val_hist_zeros.txt')
    print ("Loaded val_zeros")

if not os.path.exists(dire + 'X_test_hist_zeros.txt'):
    test_feat_zeros, test_labels_zeros = make_data_history_zeros(df_test)
    np.savetxt(dire + 'X_test_hist_zeros.txt',test_feat_zeros,fmt='%5.4f')
    np.savetxt(dire + 'Y_test_hist_zeros.txt',test_labels_zeros,fmt='%5.4f')
    print ("Saved test_zeros")
else:
    test_feat_zeros = np.loadtxt(dire + 'X_test_hist_zeros.txt')
    test_labels_zeros = np.loadtxt(dire + 'Y_test_hist_zeros.txt')
    print ("Loaded test_zeros")

Loaded train_zeros
Loaded val_zeros
Loaded test_zeros


In [16]:
x_train = train_feat_zeros
y_train = train_labels_zeros

x_val = val_feat_zeros
y_val = val_labels_zeros

x_test = test_feat_zeros
y_test = test_labels_zeros

In [17]:
x_train.shape

(156967, 200)

In [18]:
# get random sample of train/test for this model
train_idx = np.random.permutation(len(x_train))
val_idx = np.random.permutation(len(x_val))

x_train = x_train[train_idx]
y_train = y_train[train_idx]

x_val = x_val[val_idx]
y_val = y_val[val_idx]

In [19]:
# # only predict sofa
# y_train = np.expand_dims(y_train[:, 29], 1)
# y_val = np.expand_dims(y_val[:,29],1)

In [20]:
def black_box_variational_inference(logprob, D, num_samples):
    """Implements http://arxiv.org/abs/1401.0118, and uses the
    local reparameterization trick from http://arxiv.org/abs/1506.02557"""

    def unpack_params(params):
        # Variational dist is a diagonal Gaussian.
        mean, log_std = params[:D], params[D:]
        return mean, log_std

    def gaussian_entropy(log_std):
        return 0.5 * D * (1.0 + np.log(2*np.pi)) + np.sum(log_std)

    rs = npr.RandomState(0)
    def variational_objective(params, t):
        """Provides a stochastic estimate of the variational lower bound."""
        mean, log_std = unpack_params(params)
        samples = rs.randn(num_samples, D) * np.exp(log_std) + mean
        lower_bound = gaussian_entropy(log_std) + np.mean(logprob(samples, t))
        return -lower_bound

    gradient = grad(variational_objective)

    return variational_objective, gradient, unpack_params

In [39]:
def make_nn_funs(layer_sizes, L2_reg, noise_variance, nonlinearity=np.tanh):
    """These functions implement a standard multi-layer perceptron,
    vectorized over both training examples and weight samples."""
    shapes = list(zip(layer_sizes[:-1], layer_sizes[1:]))
    num_weights = sum((m+1)*n for m, n in shapes)

    def unpack_layers(weights):
        num_weight_sets = len(weights)
        for m, n in shapes:
            yield weights[:, :m*n]     .reshape((num_weight_sets, m, n)),\
                  weights[:, m*n:m*n+n].reshape((num_weight_sets, 1, n))
            weights = weights[:, (m+1)*n:]

    def predictions(weights, inputs):
        """weights is shape (num_weight_samples x num_weights)
           inputs  is shape (num_datapoints x D)"""
        inputs = np.expand_dims(inputs, 0)
        for W, b in unpack_layers(weights):
            outputs = np.einsum('mnd,mdo->mno', inputs, W) + b
            inputs = nonlinearity(outputs)
        outputs = 1.5*np.tanh(outputs)
        return outputs

    def logprob(weights, inputs, targets):
        log_prior = -L2_reg * np.sum(weights**2, axis=1)
        preds = predictions(weights, inputs)
        log_lik = -np.sum((preds - targets)**2, axis=1)[:, 0] / noise_variance
        return log_prior + log_lik

    return num_weights, predictions, logprob

In [51]:
# Specify inference problem by its unnormalized log-posterior.
from IPython import display

import pylab as pl

inputs, targets = (x_train, y_train)

rbf = lambda x: 0.6*np.exp(-x**2)
relu = lambda x: np.maximum(x, 0.01*x)
tanh = lambda x: 0.6*np.tanh(x)
num_weights, predictions, logprob = \
    make_nn_funs(layer_sizes=[x_train.shape[1], 10, 10, y_train.shape[1]], L2_reg=1.5,
                 noise_variance=0.01, nonlinearity=tanh)

batch_size = 1000

n_epoch = 5

num_batches = int(np.ceil(len(inputs) / batch_size))

glob_params = None

def batch_indices(iter):
    batchidx = iter % num_batches
    return slice(batchidx * batch_size, (batchidx+1) * batch_size)


def log_posterior(weights, t):
    cur_batch_idx = batch_indices(t)
    cur_inp = inputs[cur_batch_idx]
    cur_tar = targets[cur_batch_idx]
    return logprob(weights, cur_inp, cur_tar)

# Build variational objective.
objective, gradient, unpack_params = \
    black_box_variational_inference(log_posterior, num_weights,
                                    num_samples=50)


def callback(params, t, g):
    global glob_params
    print("Iteration {} lower bound {}".format(t, -objective(params, t)))
    
    glob_params = params

    # Sample functions from posterior.
    rs = npr.RandomState(0)
    mean, log_std = unpack_params(params)
    #rs = npr.RandomState(0)
    sample_weights = rs.randn(10, num_weights) * np.exp(log_std) + mean

    val_outputs = np.mean(predictions(sample_weights, x_val), axis=0)
#     print(val_outputs.shape)
#     import sys
#     sys.exit()
    
    mse = np.mean( (val_outputs - y_val)**2)
    
    train_idx_samp = np.random.choice(train_idx, 10000, replace=False)
    
    xt = x_train[train_idx_samp]
    yt = y_train[train_idx_samp]
    
    train_outputs = np.mean(predictions(sample_weights, xt), axis=0)
    
    mse_train = np.mean((train_outputs - yt)**2)
    
    print("Iteration {}: train mse: {} val mse: {}".format(t, mse_train, mse))
    for tup in zip(y_val[:10, 29],val_outputs[:10, 29]):
        print(tup)

#     # Plot data and functions.
#     if (t+1) % 10 == 0:
#         pl.plot(inputs.ravel(), targets.ravel(), 'bx')
#         pl.plot(plot_inputs, outputs[:, :, 0].T)
#     #     display.clear_output(wait=True)
#         pl.show()
#         print(np.squeeze(outputs).shape)

# Initialize variational parameters
rs = npr.RandomState(0)
init_mean    = rs.randn(num_weights)
init_log_std = -5 * np.ones(num_weights)
# init_mean, init_log_std = unpack_params(glob_params)
init_var_params = np.concatenate([init_mean, init_log_std])

print("Optimizing variational parameters...")
variational_params = adam(gradient, init_var_params,
                          step_size=0.05, num_iters=62, callback=callback)

Optimizing variational parameters...
Iteration 0 lower bound -149767.85633941315
Iteration 0: train mse: 1.5256512249407104 val mse: 1.522787818027553
(0.0, 1.0672272432998893)
(0.85840000000000005, -0.49900740745440597)
(-0.28610000000000002, 1.2026697416005852)
(0.28610000000000002, 0.97176297202875328)
(0.0, 1.1755986909558662)
(0.0, 1.0790898896694552)
(0.28610000000000002, 0.92085263929580174)
(-0.28610000000000002, 1.1016659791355972)
(0.0, -0.83279830558293266)
(0.28610000000000002, -0.37177154336388413)
Iteration 1 lower bound -137056.04653507943
Iteration 1: train mse: 1.4874541323482975 val mse: 1.4838963451260265
(0.0, 1.1152940945350316)
(0.85840000000000005, -0.54822525092740648)
(-0.28610000000000002, 1.1352618407560484)
(0.28610000000000002, -0.64054340172171109)
(0.0, 1.0711035504841813)
(0.0, 1.0648365238337507)
(0.28610000000000002, 1.2355105845018837)
(-0.28610000000000002, 1.1473342831628708)
(0.0, -0.90474515673214762)
(0.28610000000000002, -0.22936202047546619)
It

Iteration 17 lower bound -30513.983744649235
Iteration 17: train mse: 0.835444857889795 val mse: 0.8293873625548834
(0.0, 1.187308667135653)
(0.85840000000000005, 0.23209441028666133)
(-0.28610000000000002, 0.70933314012499205)
(0.28610000000000002, -0.56918210165468308)
(0.0, 0.98403171242066656)
(0.0, 1.0605998107234529)
(0.28610000000000002, 0.63915041802763917)
(-0.28610000000000002, 0.94938929643065839)
(0.0, -0.93606551331143917)
(0.28610000000000002, -0.41198395973215873)
Iteration 18 lower bound -28155.473528947376
Iteration 18: train mse: 0.7966539932635911 val mse: 0.791795031678807
(0.0, 1.1576070139183572)
(0.85840000000000005, 0.25821756810266727)
(-0.28610000000000002, 0.65350962653008104)
(0.28610000000000002, -0.55521359681683324)
(0.0, 0.96736118488606659)
(0.0, 1.1351869856640189)
(0.28610000000000002, 0.59378544682510648)
(-0.28610000000000002, 0.93418434858203026)
(0.0, -0.90233461890016586)
(0.28610000000000002, -0.44143822963515406)
Iteration 19 lower bound -25326

Iteration 34 lower bound -11040.849750006593
Iteration 34: train mse: 0.399539221975332 val mse: 0.39346862231194163
(0.0, 0.53858682990069884)
(0.85840000000000005, 0.47854105115579471)
(-0.28610000000000002, 0.22232547803669359)
(0.28610000000000002, -0.26433227891549049)
(0.0, 0.64867580172936712)
(0.0, 0.45303901757458276)
(0.28610000000000002, 0.41662090035101301)
(-0.28610000000000002, 0.54793120002441376)
(0.0, -0.28625589056886797)
(0.28610000000000002, -0.17931716945523851)
Iteration 35 lower bound -11137.384410325238
Iteration 35: train mse: 0.3782641480008545 val mse: 0.37875705678596694
(0.0, 0.48476306215367321)
(0.85840000000000005, 0.46813312929598949)
(-0.28610000000000002, 0.21617127932724989)
(0.28610000000000002, -0.2227259486340869)
(0.0, 0.61828793891104805)
(0.0, 0.42078367186222987)
(0.28610000000000002, 0.40888448278627243)
(-0.28610000000000002, 0.52548093520338779)
(0.0, -0.25442253042673812)
(0.28610000000000002, -0.14468664428743891)
Iteration 36 lower bound

Iteration 51 lower bound -9176.130386572291
Iteration 51: train mse: 0.23453534460161457 val mse: 0.23031843322387552
(0.0, -0.11758938171809277)
(0.85840000000000005, 0.24728786911871542)
(-0.28610000000000002, 0.071511702355456355)
(0.28610000000000002, 0.29705783083671766)
(0.0, 0.16679332395671098)
(0.0, 0.083294594864312121)
(0.28610000000000002, 0.15161119058702716)
(-0.28610000000000002, 0.21370722951495505)
(0.0, 0.1392742777428076)
(0.28610000000000002, 0.29135495164657688)
Iteration 52 lower bound -7918.556255304618
Iteration 52: train mse: 0.2236774969467112 val mse: 0.22668383971181036
(0.0, -0.11508439413885074)
(0.85840000000000005, 0.23355940857362331)
(-0.28610000000000002, 0.075871825796179751)
(0.28610000000000002, 0.30359619522050885)
(0.0, 0.15461330403767728)
(0.0, 0.08332092728753078)
(0.28610000000000002, 0.14572856626865102)
(-0.28610000000000002, 0.19981139816183821)
(0.0, 0.15565828792241049)
(0.28610000000000002, 0.29655359512868679)
Iteration 53 lower bound 

In [52]:
np.save('glob_params.npy', glob_params)

In [55]:
mean, log_std = unpack_params(glob_params)
sample_weights = rs.randn(10, num_weights) * np.exp(log_std) + mean
val_outputs = np.mean(predictions(sample_weights, x_val), axis=0)